In [6]:
import datetime
import json
import ast
import re

In [3]:
with open("verbdic_20170425.txt") as vd:
    vd_raw = vd.read()

vd_raw = re.sub("'", '"', vd_raw)
vd_raw = re.sub('False', '"False"', vd_raw)
vd_raw = re.sub('True', '"True"', vd_raw)

vd_raw = vd_raw.split("\n")

#vd_raw

In [4]:
verb_dict = []

counter = 0
for line in vd_raw:
    try:
        ll = json.loads(line)
        verb_dict.append(ll)
#         print(str(counter) , verb_dict[len(verb_dict) -1 ])
    except:
#         print(line)
        counter += 1 

In [9]:
re.findall("#(.+)#" , reconstruct_string(verb_dict[149]['verb']))
print(reconstruct_string(verb_dict[149]['verb']))

#خدم# *الطائرات العسكرية* (لقصف)


In [166]:
goal_correct_string = '#اصيب# (في انفجار)'
goal_corrupt_string = '(اصيب# (في انفجار#'
print("\n\nCorrupt :" , goal_corrupt_string)
print("\n\nCorrect :" , goal_correct_string)

print("Corrupt ascii encoding")
corrupt_encoding= goal_corrupt_string.encode('ascii', errors='ignore') 
for i in range(len(corrupt_encoding)):
    print(chr(corrupt_encoding[i]))
    
print("\n\nCorrect ascii encoding")
correct_encoding= goal_correct_string.encode('ascii', errors='ignore') 
for i in range(len(correct_encoding)):
    print(chr(correct_encoding[i]))




Corrupt : (اصيب# (في انفجار#


Correct : #اصيب# (في انفجار)
Corrupt ascii encoding
(
#
 
(
 
#


Correct ascii encoding
#
#
 
(
 
)


In [300]:
acceptable_formats = ["##()()" , "##**()"  , "##**" , "##()" , "##()**"]
#core techinque is switching the first and lat letters  
def check_and_reconstruct_verb(verb): 
    verb.strip()
    verb_encod =str(verb.encode('ascii', errors='ignore') , 'UTF8').replace(" ", "")
    if verb_encod == '' : 
        return verb
    elif verb_encod in acceptable_formats: #verb is already correct(rare cases)
        return verb 
    else : 
        arr = list(verb)
        arr= arr[-1:] + arr[:-1]
#         return "".join(arr)
            #Some verbs are entered inncorretly(coded wrong), at this point we should be able to catch them 
        reconstructed_string = "".join(arr)
        rc_string_encod = str(reconstructed_string.encode('ascii', errors='ignore') , 'UTF8').replace(" ", "").strip()
        if rc_string_encod in acceptable_formats :
            return reconstructed_string
        else :
            return reconstructed_string


In [301]:

x2= '*نشر#* قوات حفظ سلام#'
x2 = " ".join(x2.split())
# print('\n\n==')
# for i in range(len(x2)):
#     print(x2[i])
# print('\n\n==')

print(str(x2.encode('ascii', errors='ignore') , 'UTF8').replace(" ", ""))
x2 = check_and_reconstruct_verb(x2)



*#*#


In [302]:
print(x2)
re.findall("#(.+)#" , x2)

#*نشر#* قوات حفظ سلام


['*نشر']

In [162]:

arr = list('*نشر#* قوات حفظ سلام#')
#replace first char with last - op_1
#flip last char 
temp = arr[0]
arr[0] = arr[len(arr) -1]
if temp != '*' :
    arr[len(arr) -1] = ')'
else :
    arr[len(arr) -1] = temp 
    

    
print ("".join(arr))

#نشر#* قوات حفظ سلام*


## Remving duplicate rules 
Once the verbs dictionary is ready (dictionary including only verbs not synset), remove any duplicate rules 

In [69]:
verbs = [] 
with open('CAMEO.0.03_verbs_and_verb_patterns_only.txt' , 'r') as input_file:
    for i in input_file : 
        verbs.append(i.replace('\n' , ''))
verbs = [i for i in verbs if i]
verbs[:30]

['--- عمل [---] ---',
 'عمل                             # rush',
 'سرع                             # rush',
 'هرل                             # rush',
 'أنطلق                           # rush',
 'عجل                             # rush',
 'هرع                             # rush',
 '- * &MILITARY   [154] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 '- * (ل حد) [154] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 '- * (إلى حد) [154] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 '- * معونة [070] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 '- * مساعدة [070] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 '--- اضاف [---] ---',
 'اضاف                            # add',
 '- * (ل {الاحزاب الممنوعة}) [1722] # اضاف',
 '- * (ل {الاحزاب الممنوعة}) [1722] # اضاف',
 '--- نبه [---] ---',
 'نبه                             # alert',
 'نذر                             # alert',
 'حذر                             # alert',
 '- * &SECURITY (من {&WEAPON ATTACK})   [151]  # نبه,نذر,حذر',
 '--- راجع [

In [70]:
def unique(sequence):
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

In [71]:
verbs_clean = [] 
# for i in range
cr = 0 
while cr < len(verbs) : 
    if verbs[cr][:3].strip() == '---' : 
        verbs_clean.append(verbs[cr])
        cr+=1 
#         print(verbs[cr][:3] , '| ',cr)
        tmp = []
        while True:                    
            tmp.append(verbs[cr])
            cr+=1 
            if cr >= len(verbs ) or verbs[cr][:3].strip() == '---':
                break
        verbs_clean.extend(unique(tmp))
        verbs_clean.append(' ')
        verbs_clean.append(' ')
        
        
        

In [72]:
verbs_clean[:30]

['--- عمل [---] ---',
 'عمل                             # rush',
 'سرع                             # rush',
 'هرل                             # rush',
 'أنطلق                           # rush',
 'عجل                             # rush',
 'هرع                             # rush',
 '- * &MILITARY   [154] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 '- * (ل حد) [154] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 '- * (إلى حد) [154] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 '- * معونة [070] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 '- * مساعدة [070] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 ' ',
 ' ',
 '--- اضاف [---] ---',
 'اضاف                            # add',
 '- * (ل {الاحزاب الممنوعة}) [1722] # اضاف',
 ' ',
 ' ',
 '--- نبه [---] ---',
 'نبه                             # alert',
 'نذر                             # alert',
 'حذر                             # alert',
 '- * &SECURITY (من {&WEAPON ATTACK})   [151]  # نبه,نذر,حذر',
 ' ',
 ' ',
 '--- راجع [---] ---'

In [74]:
print(len(verbs))
print(len(verbs_clean))
print('really ? ......')

4804
4808
really ? ......


In [76]:
with open('verbs_no_duplicates.txt' , 'w') as out : 
    for i in verbs_clean : 
        out.write('{}\n'.format(i))

In [61]:
x = ['x' , 'x' , 'y']
y = [i for i in x if x.count(i)==1]
y

['y']

In [62]:

unique(x)

['x', 'y']

### Extracting Entires related to sepcific verb code

In [2]:
verbs = [] 
with open('untitled.txt' , 'r') as input_file:
    for i in input_file : 
        verbs.append(i.replace('\n' , ''))
verbs = [i for i in verbs if i]
verbs[:30]

['--- عمل [---] ---',
 'عمل                             # rush',
 'سرع                             # rush',
 'هرل                             # rush',
 'أنطلق                           # rush',
 'عجل                             # rush',
 'هرع                             # rush',
 '- * &MILITARY   [154] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 '- * (ل حد) [154] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 '- * (إلى حد) [154] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 '- * معونة [070] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 '- * مساعدة [070] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 ' ',
 ' ',
 '--- اضاف [---] ---',
 'اضاف                            # add',
 '- * (ل {الاحزاب الممنوعة}) [1722] # اضاف',
 ' ',
 ' ',
 '--- نبه [---] ---',
 'نبه                             # alert',
 'نذر                             # alert',
 'حذر                             # alert',
 '- * &SECURITY (من {&WEAPON ATTACK})   [151]  # نبه,نذر,حذر',
 ' ',
 ' ',
 '--- راجع [---] ---'

In [97]:
## helper methods
def unique(sequence):
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

def check_entry(entry, code) :
    for i in entry : 
        ex_code = re.findall("\[(.*?)\]", i)
#         print('checking : {} | extracted_code : {} | tagert_code : {}'.format(i , ex_code, target_code))
        if not ex_code : 
            continue
        if ex_code[0].startswith(code) : 
#             print('it does')
            return True
    return False
    
def clean_entry(entry, code ) :  # returns entry with on roles that have the sepcific code 
    new_entry = [] 
    new_entry.append(entry[0])
    for i in range(1, len(entry)) :
        if entry[i].startswith('- *') : 
            ex_code = re.findall("\[(.*?)\]", entry[i])
            if not ex_code : 
                continue
            if ex_code[0].startswith(code) : 
    #             print('it does')
                new_entry.append(entry[i])
        else : 
            new_entry.append(entry[i])
    return new_entry
        

In [107]:
target_verbs = []
target_code = '14'

# for i in range
cr = 0 
while cr < len(verbs) : 
    entry = []
    if verbs[cr][:3].strip() == '---' : 
        entry.append(verbs[cr])
        cr+=1 
#         print(verbs[cr][:3] , '| ',cr)
#         tmp = []
        while True:                    
            entry.append(verbs[cr])
            cr+=1 
            if cr >= len(verbs ) or verbs[cr][:3].strip() == '---':
                break
        contains_verb = check_entry(entry , target_code)
#         print (unique(entry))
        if contains_verb : 
            target_verbs.extend(clean_entry(unique(entry), target_code))
            target_verbs.append(' ') # aesthetic stuff 
            target_verbs.append(' ')
         
        
            

In [110]:
# target_verbs

In [111]:
# outputting 
with open('verbs_assualt_only.txt' , 'w') as out : 
    for i in target_verbs : 
        out.write('{}\n'.format(i))

In [104]:
target_verbs

['--- عمل [---] ---',
 'عمل                             # rush',
 'سرع                             # rush',
 'هرل                             # rush',
 'أنطلق                           # rush',
 'عجل                             # rush',
 'هرع                             # rush',
 '- * &MILITARY   [154] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 '- * (ل حد) [154] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 '- * (إلى حد) [154] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 ' ',
 ' ',
 ' ']

In [98]:
clean_entry(entry , '15')

['--- عمل [---] ---',
 'عمل                             # rush',
 'سرع                             # rush',
 'هرل                             # rush',
 'أنطلق                           # rush',
 'عجل                             # rush',
 'هرع                             # rush',
 '- * &MILITARY   [154] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 '- * (ل حد) [154] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 '- * (إلى حد) [154] # عمل_بسرعة,سرع,هرل,اِنْطلق_بِسُرْعة,عجل,هرع',
 ' ',
 ' ']